[Manual](https://python-docx.readthedocs.io/en/latest/index.html)

# Установка и импорт библиотек

In [2]:
import pandas as pd
import numpy as np

In [2]:
!pip install python-docx

  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184606 sha256=f9635461b13d12bcb09aa583324b6881fd1ed2e3f46372e3efa39bacd1ff2f18
  Stored in directory: c:\users\yupes\appdata\local\pip\cache\wheels\32\b8\b2\c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [4]:
from docx import Document

In [5]:
#логер
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

# Конвертация

In [7]:
document = Document('../data/Reestr ATY 01 2020 (1).docx')

In [45]:
data = {1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
for table in log_progress(document.tables, 1):
    for i,row in enumerate(table.rows):
        count = 8
        for cell in log_progress(row.cells, 1):
#             print(count)
            if cell.text.strip() == 'Сведения':
                count = 1
            if count < 8:
                data[count].append(cell.text)
#                 for paragraph in cell.paragraphs:
#                     for run in paragraph.runs:
#                         if run.bold:
#                             print(run.text)
                count+=1

In [56]:
objects = []
for table in log_progress(document.tables, 1):
    for i,row in enumerate(table.rows):
        objects.append([cell.text for cell in row.cells if row.cells[0].text == 'Сведения'])

In [58]:
len_objects = []
[len_objects.append(len(o)) for o in objects]
set(len_objects)

{0, 5, 6, 7, 8, 9}

In [59]:
[o for o in objects if len(o)== 9]

[['Сведения',
  '',
  'Иркутский',
  'Иркутский',
  'г. Иркутск',
  'г. Иркутск',
  'Иркутское районное муниципальное образование',
  '25212000000',
  '0']]

In [60]:
objects.append([['Сведения',
  '',
  'Иркутский',
  'г. Иркутск',
  'Иркутское районное муниципальное образование',
  '25212000000',
  '0']])

In [61]:
[o for o in objects if len(o)== 8]

[['Сведения',
  '',
  'Баяндаевский ',
  'с. Баяндай',
  'с. Баяндай',
  'Муниципальное образование «Баяндаевский район»',
  '25125000000',
  '132'],
 ['Сведения',
  '',
  'Боханский',
  'п. Бохан',
  'п. Бохан',
  'Муниципальное образование «Боханский район»',
  '25126000000',
  '124'],
 ['Сведения',
  '',
  'Куйтунский',
  'р.п. Куйтун',
  'Муниципальное образование Куйтунский район',
  '25222000000',
  '312',
  'Реквизиты правового акта (иного документа)'],
 ['Сведения',
  '',
  'Мамско-Чуйский',
  'р.п. Мама',
  'Муниципальное образование Мамско-Чуйского района',
  'Муниципальное образование Мамско-Чуйского района',
  '25224000000',
  '1000\nвоздушным путем']]

In [62]:
objects.extend([['Сведения',
  '',
  'Баяндаевский ',
  'с. Баяндай',
  'Муниципальное образование «Баяндаевский район»',
  '25125000000',
  '132'],
 ['Сведения',
  '',
  'Боханский',
  'п. Бохан',
  'Муниципальное образование «Боханский район»',
  '25126000000',
  '124'],
 ['Сведения',
  '',
  'Куйтунский',
  'р.п. Куйтун',
  'Муниципальное образование Куйтунский район',
  '25222000000',
  '312'],
 ['Сведения',
  '',
  'Мамско-Чуйский',
  'р.п. Мама',
  'Муниципальное образование Мамско-Чуйского района',
  '25224000000',
  '1000 воздушным путем']])

In [64]:
for i in [o for o in objects if len(o)== 6]:
    objects.append([i[0], '', i[2], 'город', i[3], i[4], i[5]])

In [68]:
for i in [o for o in objects if len(o)== 5]:
    objects.append([i[0], '', i[2], 'район', i[3], i[4], 0])

In [82]:
df_dict = {'name':[], 'type':[], 'name_MO':[], 'code':[], 'distance':[]}
for o in objects:
    if len(o)==7:
        df_dict['name'].append(o[2])
        df_dict['type'].append(o[3].replace('\n', ' '))
        df_dict['name_MO'].append(o[4])
        df_dict['code'].append(o[5])
        df_dict['distance'].append(o[6])

In [84]:
df = pd.DataFrame(df_dict)

In [85]:
df.to_csv('../data/localities.csv', index = False)

## Названия полей 
```
name     - Наименование населенного пункта Иркутской области
type     - Вид и категория населенного пункта Иркутской области
name_MO  - Наименование муниципального образования, в границах которого образован населенный пункт
code     - Код населенного пункта Иркутской области как объекта ОКАТО
distance - Расстояние до административного центра района Иркутской области
```